In [1]:
import tensorflow as tf
from tensorflow import keras
import os
# Chay bang CPU thay vi GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

class buildmodel():
    def conv_block(self,inputs , filters, block_num,conv_num=2):
        x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_1'%block_num)(inputs)
        x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_2'%block_num)(x)
        if conv_num==3:
            x = keras.layers.Conv2D(filters,3, activation = 'relu', padding = 'same',name = 'conv_%d_3'%block_num)(x)
        x = keras.layers.MaxPooling2D(name = 'max_pooling_%d'%block_num)(x)
        return x

    def vgg16(self,num_outputs=1000):
        inputs = keras.layers.Input(shape=(224,224,3),name = 'input')
        x = self.conv_block(inputs,64,1)
        x = self.conv_block(x,128,2)
        x = self.conv_block(x,256,3,3)
        x = self.conv_block(x,512,4,3)
        x = self.conv_block(x,512,5,3)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_1')(x)
        x = keras.layers.Dense(4096,activation = 'relu',name = 'dense_2')(x)
        outputs = keras.layers.Dense(num_outputs,activation = 'softmax',name = 'outputs')(x)
        model = keras.Model(inputs = inputs, outputs = outputs, name = 'vgg16')
        return model

    def residual_block(self,inputs, filter,block_num,reduce=False):
        res = inputs
        x = keras.layers.Conv2D(filter,(3, 3), strides = 2 if reduce else 1,activation = 'relu', padding = 'same',name = f'conv_{block_num}_1')(res)
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_2')(x)
        if reduce == True:
            res = keras.layers.Conv2D(filter,(1, 1), strides = 2, activation = 'relu', padding = 'same',name = f'conv_skip_{block_num-1}')(res)
        res = keras.layers.Add(name = f'end_res_bock_{2*block_num-1}')([x,res])
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_3')(res)
        x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_4')(x)
        res = keras.layers.Add(name = f'end_res_bock_{2*block_num}')([x,res])
        return res
        
    def resnet18(self,num_outputs=1000):
        # Inputs
        inputs = keras.layers.Input(shape=(224,224,3),name = 'input')
        x = keras.layers.Conv2D(64,(7, 7),strides = 2, activation = 'relu', padding = 'same',name = 'conv2D')(inputs)
        x = keras.layers.MaxPooling2D(pool_size = (3,3),strides = 2 ,padding = 'same',name = 'max_pool_1')(x)
        # Block 1
        x = self.residual_block(x,64,1,False)
        # Block 2
        x = self.residual_block(x,128,2,True)
        # Block 3
        x = self.residual_block(x,256,3,True)
        # Block 4
        x = self.residual_block(x,512,4,True)
        # Ouputs
        x = keras.layers.GlobalAveragePooling2D(name = 'global_pooling')(x)
        if num_outputs==1:
            outputs = keras.layers.Dense(num_outputs,activation = 'sigmoid',name = 'outputs_sigmoid')(x)
        else:
            outputs = keras.layers.Dense(num_outputs,activation = 'softmax',name = 'outputs_softmax')(x)
        model = keras.Model(inputs = inputs, outputs = outputs, name = 'resnet18')
        return model
    def compiled_model(self,model_name = 'resnet18',num_outputs=1):
        if model_name == 'resnet18':
            model = self.resnet18(num_outputs = num_outputs)
        if model_name == 'vgg16':
            model = self.vgg16(num_outputs=num_outputs)
        model.compile(optimizer=keras.optimizers.Adam(),loss=keras.losses.BinaryCrossentropy())
        return model
    def __init__(self,model_name = 'resnet18',num_outputs=1):
        if model_name == 'resnet18':
            self.model = self.resnet18(num_outputs = num_outputs)
        if model_name == 'vgg16':
            self.model = self.vgg16(num_outputs=num_outputs)
        self.opt = keras.optimizers.Adam()
        self.loss_fn = keras.losses.BinaryCrossentropy()
        self.metric = keras.metrics.BinaryAccuracy()
    @tf.function
    def train_step(self,x, y):
        with tf.GradientTape() as tape:
            logits = self.model(x, training=True)
            loss_value = self.loss_fn(y, logits)
        grads = tape.gradient(loss_value, self.model.trainable_weights)
        self.metric.update_state(y,logits)
        return grads        
    @tf.function
    def update_grads(self, grads):
        self.opt.apply_gradients(zip(grads, self.model.trainable_weights))

2023-12-06 15:12:19.054958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-06 15:12:19.176972: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 15:12:19.714965: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/pai004/anaconda3/lib/:/home/pai004/anaconda3/lib/python3.9/site-packages/nvidia/cudnn/lib:/home/pai004/anaconda3/envs/tan/lib/
2023-12-06 15:12:19.715030: W tensorflow/stream_executor/platform/default/dso_

In [2]:
model = buildmodel().model

2023-12-06 15:12:20.613124: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-12-06 15:12:20.613165: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: pai004
2023-12-06 15:12:20.613175: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: pai004
2023-12-06 15:12:20.613293: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.76.0
2023-12-06 15:12:20.613328: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.76.0
2023-12-06 15:12:20.613337: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.76.0
2023-12-06 15:12:20.613636: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perf

In [3]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_gen = img_gen.flow_from_directory('./train',target_size=(224, 224),class_mode="binary")

Found 10028 images belonging to 2 classes.


In [5]:
import time

epochs = 1
for epoch in range(epochs):
    model = buildmodel()
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_gen):
        print(step)
        grads = model.train_step(x_batch_train, y_batch_train)
        model.update_grads(grads)
        print(model.metric.result())

    # # Display metrics at the end of each epoch.
    # train_acc = train_acc_metric.result()
    # print("Training acc over epoch: %.4f" % (float(train_acc),))

    # # Reset training metrics at the end of each epoch
    # train_acc_metric.reset_states()

    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
0
tf.Tensor(0.0625, shape=(), dtype=float32)
1
tf.Tensor(0.484375, shape=(), dtype=float32)
2
tf.Tensor(0.625, shape=(), dtype=float32)
3
tf.Tensor(0.6875, shape=(), dtype=float32)
4
tf.Tensor(0.7375, shape=(), dtype=float32)
5
tf.Tensor(0.7395833, shape=(), dtype=float32)
6
tf.Tensor(0.7589286, shape=(), dtype=float32)
7
tf.Tensor(0.75, shape=(), dtype=float32)
8
tf.Tensor(0.7465278, shape=(), dtype=float32)
9


KeyboardInterrupt: 